In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import nibabel as nb
import hrf_tools
import hcp_utils as hcp
from analysis import plot_results

import matplotlib.pyplot as plt

/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
import numpy as np
import nibabel as nb
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from nilearn_plotting_custom import plot_surf
from PIL import Image
from PIL import ImageDraw
import npp
import hcp_utils as hcp
from hcp_tools import load_flatmaps_59k
from hcp_tools import load_meshes
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
sns.set("paper", "white")
#%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

def plot_results(scores,score_type,data_type,vertex_info,subject,feature,dataset,title):
    '''Inputs:
        scores = data to plot
        score_type = r2, r, p, z, d, raw
        data_type = 32k (3T) or 59k (7T)
        vertex info = None or the vertex info if it is 59k data beacuse hcp_utils doesnt by default
        subject = eg 100610 subject id for file naming
        feature = eg as_scores plotted feature for file naming
        dataset = eg merlin or HCP_7T which dataset?
        title = 
    '''
    scratch_dir = '../tmp'
#     scratch_dir = '/scratch/scratch/Fri/jsmentch/tmp'
#     if not os.path.exists(scratch_dir):
#         os.mkdir(scratch_dir)
    if score_type == 'r2':
        v=[0,0.5]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'r':
        v=[0,1]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'p':
        v=[0,0.05]
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'z':
        v=[-10,10]
        threshold=3
        symmetric_cmap=True
        cmap='cold_hot'
    if score_type == 'd':
        v=[0,10]
        threshold=3
        symmetric_cmap=True
        cmap='inferno'
    if score_type == 'raw':
        v=[-10,10]
        threshold=None
        symmetric_cmap=True
        cmap='cold_hot'
    save_dir=f'../outputs/figures/{dataset}/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)    
    if data_type == '59k':
        flatmeshes=load_flatmaps_59k() #load flatmaps
        surf_path_msm = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.L.inflated_1.6mm_MSMAll.59k_fs_LR.surf.gii'
        mesh59k_msm = load_meshes(example_filename=surf_path_msm) #load other meshes
        # get data from results in plotting format
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0, vertex_info=vertex_info)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0, vertex_info=vertex_info)
        # sulcal depth paths
        sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
        sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
        # params for view to plot
        params = [('flat_L',score_cortex_dataL,flatmeshes.flat_left,sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,flatmeshes.flat_right,sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,mesh59k_msm.very_inflated_left,sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,mesh59k_msm.very_inflated_right,sulc_right,'right'),\
        ]
    elif data_type == '32k':
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0)
    #     # sulcal depth paths
    #     sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
    #     sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
    #     # params for view to plot
        params = [('flat_L',score_cortex_dataL,hcp.mesh.flat_left,hcp.mesh.sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,hcp.mesh.flat_right,hcp.mesh.sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,hcp.mesh.very_inflated_left,hcp.mesh.sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,hcp.mesh.very_inflated_right,hcp.mesh.sulc_right,'right'),\
        ]
    # plot each hemi and mesh, save to outputs dir
    for name, data, mesh, sulc, hemi in params:
        #figure, axes = plt.subplots(subplot_kw=dict(projection="3d"), figsize=(6,4))
        plot_surf(mesh,\
                data, \
                  cmap=cmap,symmetric_cmap=symmetric_cmap, avg_method='median',#figure=fig,\
                bg_map=sulc, colorbar=True, vmin=v[0], vmax=v[1], threshold=threshold, hemi=hemi, \
                #data_alpha=np.where(data>0,1,0),\
                data_alpha=np.ones(data.shape),\
                data_remove=np.zeros(data.shape),output_file=f'{scratch_dir}/{name}.png')
#combine saved maps into one with PIL
#     if notebook==True:
    area = (75, 140, 635, 560) #area to crop from each image
#     else:
#         area = (105, 190, 880, 780)
        
    img = Image.open(f'{scratch_dir}/flat_L.png',mode='r')
    img = img.resize((770,720))
    cropped = img.crop(area)
    fL=cropped.transpose(Image.FLIP_LEFT_RIGHT)
    w,h = img.size
    c_area = (690, 0, w-10, h) # area of colorbar to crop
    cbar = img.crop(c_area)

    img = Image.open(f'{scratch_dir}/flat_R.png',mode='r')
    img = img.resize((770,720))
    fR = img.crop(area)

    img = Image.open(f'{scratch_dir}/vinf_L.png',mode='r')
    img = img.resize((770,720))
    iL = img.crop(area)
    #iL=cropped.transpose(Image.FLIP_LEFT_RIGHT)

    img = Image.open(f'{scratch_dir}/vinf_R.png',mode='r')
    img = img.resize((770,720))
    iR = img.crop(area)

    w,h=iR.size

    new_im = Image.new('RGB', ( (w*2)+70 , h*2) ,(255, 255, 255, 1))
    new_im.paste(fL,(0,h))
    new_im.paste(fR,(w,h))
    new_im.paste(iL,(0,0))
    new_im.paste(iR,(w,0))
    new_im.paste(cbar,(w*2,int(round(h/4))))

    w,h=new_im.size

    draw = ImageDraw.Draw(new_im)
    draw.text((0,0),f"{title}_{subject}_{feature}_{score_type}",(0,0,0))

    new_im.save(f'{save_dir}{title}_{subject}_{feature}_{score_type}.png')
#     os.remove(f'{scratch_dir}/flat_L.png')
#     os.remove(f'{scratch_dir}/flat_R.png')
#     os.remove(f'{scratch_dir}/vinf_L.png')
#     os.remove(f'{scratch_dir}/vinf_R.png')

In [3]:
scaler = StandardScaler()

In [4]:
im_file = f'../sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/derivatives/cleaned/smoothed/sub-sid000005_run1_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii'

#im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
img = nb.load(im_file)
img_y = img.get_fdata()
img_y[np.isnan(img_y)] = 0
Y=img_y
#Y = scaler.fit_transform(img_y)

In [5]:
img_y_mean = img_y.mean(axis=0)
Y_mean = Y.mean(axis=0)

In [6]:
vertex_info = hcp.get_HCP_vertex_info(img)

In [ ]:
import gc
import os
for n in np.arange(598):
    if not os.path.isfile(f'../outputs/figures/budapest/plot_gif{n}_sid000005_raw_raw.png'):
        print(n)
        plot_results(Y[n],'raw','32k',vertex_info=vertex_info,subject='sid000005',feature='raw', dataset='budapest',title=f'plot_gif{n}' )
    gc.collect()

## they were numbered poorly, fix the numberings

In [14]:
import gc
import os
for n in np.arange(598):
    os.rename(f'../outputs/figures/budapest/plot_gif{n}_sid000005_raw_raw.png',f'../outputs/figures/budapest/plot_gif{n:03d}_sid000005_raw_raw.png')


In [6]:
from PIL import Image, ImageDraw
import os

img1 = Image.open(f'../outputs/figures/budapest/plot_gif0_sid000005_raw_raw.png')

append_images=[]
for i in np.arange(1,597):
    img2 = Image.open(f'../outputs/figures/budapest/plot_gif{i}_sid000005_raw_raw.png')
    append_images.append(img2)
img1.save(f'../outputs/figures/budapest/budapest_sid000005.gif', save_all=True, append_images=append_images, duration=0.1, loop=0)